## Group6 Project

Group Member: Weihang Wen, Mengchen Xiao, Qiao Zhang, Muchen Zhu

Kaggle Competition: Predict Future Sales https://www.kaggle.com/c/competitive-data-science-predict-future-sales

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
import time
import gzip
import seaborn as sns
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm

In [2]:
Validation = False

In [3]:
item_cat = pd.read_csv('item_categories.csv')
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [4]:
item = pd.read_csv('items.csv')
item.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
shops = pd.read_csv('shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [6]:
def get_csv(data):
    f = gzip.open(data, 'rb')
    data_content = pd.read_csv(f)
    return data_content

In [7]:
train = get_csv('sales_train.csv.gz')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [8]:
test = get_csv('test.csv.gz')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [9]:
sample_submission = get_csv('sample_submission.csv.gz')
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


### Outliers

In [10]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=train.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price)

In [11]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

There is one item with price below zero. Impute it with median.

In [12]:
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

Several shops are duplicates of each other (according to its name). Fix train and test set.

In [13]:
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57

train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58

train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

### Data Preprocessing

Find the unique variables in test and apply them in train dataset. Change the date to date format.

In [14]:
test_nrow = test.shape[0]

train = train.merge(test[['shop_id']].drop_duplicates(), how = 'inner')

train['date'] = pd.to_datetime(train['date'], format = '%d.%m.%Y')


In [15]:
train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.0,1.0
1,2013-01-10,0,59,22151,399.0,1.0
2,2013-01-04,0,59,5603,699.0,1.0
3,2013-01-19,0,59,5587,199.0,2.0
4,2013-01-31,0,59,5613,5571.0,1.0
5,2013-01-10,0,59,5623,699.0,1.0
6,2013-01-14,0,59,5623,699.0,1.0
7,2013-01-10,0,59,5629,2390.0,1.0
8,2013-01-04,0,59,5643,2390.0,1.0
9,2013-01-17,0,59,5643,2390.0,2.0


Find each item sold in every shop in every month.

In [16]:
grid = []

for block_num in train['date_block_num'].unique():

     cur_shops = train[train['date_block_num']==block_num]['shop_id'].unique()

     cur_items = train[train['date_block_num']==block_num]['item_id'].unique()

     grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

In [17]:
#turn the grid into pandas dataframe
index_cols = ['shop_id', 'item_id', 'date_block_num']

grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [18]:
grid.head(10)

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,22151,0
2,59,5603,0
3,59,5587,0
4,59,5613,0
5,59,5623,0
6,59,5629,0
7,59,5643,0
8,59,5994,0
9,59,5992,0


After checking the quantile of the 'item_cnt_day', we control it from 0 to 20.

In [19]:
train['item_cnt_day'].quantile([.9,.95,.99])

0.90    2.0
0.95    2.0
0.99    5.0
Name: item_cnt_day, dtype: float64

In [20]:
train['item_cnt_day'] = train['item_cnt_day'].clip(0,20)

Group by 'shop_id', 'item_id', 'date_block_num', we calculate the number of item sold in one month.

In [21]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
gb_cnt = train.groupby(index_cols)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})

In [22]:
gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)

In [23]:
def downcast_dtypes(df):

    float_cols = [c for c in df if df[c].dtype == "float64"]

    int_cols =   [c for c in df if df[c].dtype in ["int64", "int32"]]



    df[float_cols] = df[float_cols].astype(np.float32)

    df[int_cols]   = df[int_cols].astype(np.int16)



    return df

In [24]:
#join aggregated data to the grid to find the item sold and unsold in every shop in every month.
train1 = pd.merge(grid,gb_cnt,how='left',on=index_cols).fillna(0)

In [25]:
train1.head(10)

,shop_id,item_id,date_block_num,item_cnt_month
0,59,22154,0,1.0
1,59,22151,0,2.0
2,59,5603,0,1.0
3,59,5587,0,2.0
4,59,5613,0,1.0
5,59,5623,0,2.0
6,59,5629,0,1.0
7,59,5643,0,4.0
8,59,5994,0,1.0
9,59,5992,0,1.0


In [26]:
train1['item_cnt_month'] = train1['item_cnt_month'].astype(int)
train1 = downcast_dtypes(train1)

In [27]:
train1.head(10)

,shop_id,item_id,date_block_num,item_cnt_month
0,59,22154,0,1
1,59,22151,0,2
2,59,5603,0,1
3,59,5587,0,2
4,59,5613,0,1
5,59,5623,0,2
6,59,5629,0,1
7,59,5643,0,4
8,59,5994,0,1
9,59,5992,0,1


In [28]:
train1.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [29]:
train1.head(10)

,shop_id,item_id,date_block_num,item_cnt_month
85820,2,19,0,0
88148,2,27,0,1
90860,2,28,0,0
89310,2,29,0,0
85240,2,32,0,0
85812,2,33,0,1
85813,2,34,0,0
85239,2,35,0,0
89321,2,40,0,0
88897,2,41,0,0


In [30]:
#join item category into train and test
train1 = train1.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')
test = test.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')

In [31]:
train1.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id
0,2,19,0,0,40
1,2,27,0,1,19
2,2,28,0,0,30
3,2,29,0,0,23
4,2,32,0,0,40


In [32]:
test.head()

,ID,shop_id,item_id,item_category_id
0,0,5,5037,19
1,1,5,5320,55
2,2,5,5233,19
3,3,5,5232,23
4,4,5,5268,20


Translate item category names

In [33]:
l_cat = list(item_cat.item_category_name)

for ind in range(0,1):

     l_cat[ind] = 'PC Headsets / Headphones'

for ind in range(1,8):

     l_cat[ind] = 'Access'

l_cat[8] = 'Tickets (figure)'

l_cat[9] = 'Delivery of goods'

for ind in range(10,18):

     l_cat[ind] = 'Consoles'

for ind in range(18,25):

    l_cat[ind] = 'Consoles Games'

l_cat[25] = 'Accessories for games'

for ind in range(26,28):

    l_cat[ind] = 'phone games'

for ind in range(28,32):

    l_cat[ind] = 'CD games'

for ind in range(32,37):

    l_cat[ind] = 'Card'

for ind in range(37,43):

    l_cat[ind] = 'Movie'

for ind in range(43,55):

    l_cat[ind] = 'Books'

for ind in range(55,61):

    l_cat[ind] = 'Music'

for ind in range(61,73):

    l_cat[ind] = 'Gifts'

for ind in range(73,79):

    l_cat[ind] = 'Soft'

for ind in range(79,81):

    l_cat[ind] = 'Office'

for ind in range(81,83):

    l_cat[ind] = 'Clean'

l_cat[83] = 'Elements of a food'

In [34]:
l_cat

['PC Headsets / Headphones',
 'Access',
 'Access',
 'Access',
 'Access',
 'Access',
 'Access',
 'Access',
 'Tickets (figure)',
 'Delivery of goods',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Accessories for games',
 'phone games',
 'phone games',
 'CD games',
 'CD games',
 'CD games',
 'CD games',
 'Card',
 'Card',
 'Card',
 'Card',
 'Card',
 'Movie',
 'Movie',
 'Movie',
 'Movie',
 'Movie',
 'Movie',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Music',
 'Music',
 'Music',
 'Music',
 'Music',
 'Music',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Soft',
 'Soft',
 'Soft',
 'Soft',
 'Soft',
 'Soft',
 'Office',
 'Office',
 'Clean',
 'Clean',
 'E

Encoding item category by label encoding

In [35]:
from sklearn import preprocessing

lb = preprocessing.LabelEncoder()

item_cat['item_cat_id_fix'] = lb.fit_transform(l_cat)

In [36]:
item_cat

,item_category_name,item_category_id,item_cat_id_fix
0,PC - Гарнитуры/Наушники,0,14
1,Аксессуары - PS2,1,0
2,Аксессуары - PS3,2,0
3,Аксессуары - PS4,3,0
4,Аксессуары - PSP,4,0
5,Аксессуары - PSVita,5,0
6,Аксессуары - XBOX 360,6,0
7,Аксессуары - XBOX ONE,7,0
8,Билеты (Цифра),8,16
9,Доставка товара,9,8


Combine label encoding into train and test

In [37]:
train1 = train1.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')

test = test.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')

In [38]:
test.head()

,ID,shop_id,item_id,item_category_id,item_cat_id_fix
0,0,5,5037,19,7
1,1,5,5320,55,12
2,2,5,5233,19,7
3,3,5,5232,23,7
4,4,5,5268,20,7


In [39]:
lst = [item, item_cat, grid, gb_cnt]
del lst

gc.collect()

357

Using target encoding to encode 'shop_id', 'item_id', 'item_category_id', 'item_cat_id_fix' by four encoding methods and choosing the best one.

In [40]:
Target = 'item_cnt_month'

global_mean =  train1[Target].mean()

y_tr = train1[Target].values

In [41]:
mean_encoded_col = ['shop_id', 'item_id', 'item_category_id', 'item_cat_id_fix']

for col in tqdm(mean_encoded_col):



    col_tr = train1[[col] + [Target]]

    corrcoefs = pd.DataFrame(columns = ['Cor'])



    # 3.1.1 Mean encodings - KFold scheme
    SEED = 0

    from sklearn.model_selection import KFold

    kf = KFold(n_splits = 5, shuffle = False, random_state = SEED)



    col_tr[col + '_cnt_month_mean_Kfold'] = global_mean

    for tr_ind, val_ind in kf.split(col_tr):

        X_tr, X_val = col_tr.iloc[tr_ind], col_tr.iloc[val_ind]

        means = X_val[col].map(X_tr.groupby(col)[Target].mean())

        X_val[col + '_cnt_month_mean_Kfold'] = means

        col_tr.iloc[val_ind] = X_val

    col_tr.fillna(global_mean, inplace = True)

    corrcoefs.loc[col + '_cnt_month_mean_Kfold'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Kfold'])[0][1]



    # 3.1.2 Mean encodings - Leave-one-out scheme

    item_id_target_sum = col_tr.groupby(col)[Target].sum()

    item_id_target_count = col_tr.groupby(col)[Target].count()

    col_tr[col + '_cnt_month_sum'] = col_tr[col].map(item_id_target_sum)

    col_tr[col + '_cnt_month_count'] = col_tr[col].map(item_id_target_count)

    col_tr[col + '_target_mean_LOO'] = (col_tr[col + '_cnt_month_sum'] - col_tr[Target]) / (col_tr[col + '_cnt_month_count'] - 1)

    col_tr.fillna(global_mean, inplace = True)

    corrcoefs.loc[col + '_target_mean_LOO'] = np.corrcoef(y_tr, col_tr[col + '_target_mean_LOO'])[0][1]





    # 3.1.3 Mean encodings - Smoothing

    item_id_target_mean = col_tr.groupby(col)[Target].mean()

    item_id_target_count = col_tr.groupby(col)[Target].count()

    col_tr[col + '_cnt_month_mean'] = col_tr[col].map(item_id_target_mean)

    col_tr[col + '_cnt_month_count'] = col_tr[col].map(item_id_target_count)

    alpha = 100

    col_tr[col + '_cnt_month_mean_Smooth'] = (col_tr[col + '_cnt_month_mean'] *  col_tr[col + '_cnt_month_count'] + global_mean * alpha) / (alpha + col_tr[col + '_cnt_month_count'])

    col_tr[col + '_cnt_month_mean_Smooth'].fillna(global_mean, inplace=True)

    corrcoefs.loc[col + '_cnt_month_mean_Smooth'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Smooth'])[0][1]





   # 3.1.4 Mean encodings - Expanding mean scheme

    cumsum = col_tr.groupby(col)[Target].cumsum() - col_tr[Target]

    sumcnt = col_tr.groupby(col).cumcount()

    col_tr[col + '_cnt_month_mean_Expanding'] = cumsum / sumcnt

    col_tr[col + '_cnt_month_mean_Expanding'].fillna(global_mean, inplace=True)

    corrcoefs.loc[col + '_cnt_month_mean_Expanding'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Expanding'])[0][1]



    train1 = pd.concat([train1, col_tr[corrcoefs['Cor'].idxmax()]], axis = 1)

    print(corrcoefs.sort_values('Cor'))



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xiaom\Downloads\anaconda\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

                                       Cor
shop_id_cnt_month_mean_Kfold      0.173306
shop_id_target_mean_LOO           0.175441
shop_id_cnt_month_mean_Smooth     0.175467
shop_id_cnt_month_mean_Expanding  0.175598


 25%|█████████████████████                                                               | 1/4 [00:13<00:41, 13.83s/it]C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                                       Cor
item_id_cnt_month_mean_Kfold      0.312751
item_id_cnt_month_mean_Smooth     0.479286
item_id_target_mean_LOO           0.481372
item_id_cnt_month_mean_Expanding  0.565389


 50%|██████████████████████████████████████████                                          | 2/4 [00:32<00:32, 16.47s/it]C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                                                Cor
item_category_id_cnt_month_mean_Kfold      0.270223
item_category_id_cnt_month_mean_Smooth     0.289990
item_category_id_target_mean_LOO           0.290020
item_category_id_cnt_month_mean_Expanding  0.292934


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:46<00:15, 15.44s/it]C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xiaom\Downloads\anaconda\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


                                               Cor
item_cat_id_fix_cnt_month_mean_Kfold      0.157378
item_cat_id_fix_target_mean_LOO           0.171613
item_cat_id_fix_cnt_month_mean_Smooth     0.171658
item_cat_id_fix_cnt_month_mean_Expanding  0.176918


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:59<00:00, 14.94s/it]


### Feature Engineering

In [42]:
# Combine trainset and testset 

if Validation == False:

    test['date_block_num'] = 34

    all_data = pd.concat([train1, test], axis = 0)

    all_data = all_data.drop('ID', axis=1)

else:

    all_data = train1


del train1, test, col_tr

gc.collect()


all_data = downcast_dtypes(all_data)

In [43]:
all_data.head()

,date_block_num,item_cat_id_fix,item_cat_id_fix_cnt_month_mean_Expanding,item_category_id,item_category_id_cnt_month_mean_Expanding,item_cnt_month,item_id,item_id_cnt_month_mean_Expanding,shop_id,shop_id_cnt_month_mean_Expanding
0,0,11,0.311029,40,0.311029,0.0,19,0.311029,2,0.311029
1,0,7,0.311029,19,0.311029,1.0,27,0.311029,2,0.000000
2,0,3,0.311029,30,0.311029,0.0,28,0.311029,2,0.500000
3,0,7,1.000000,23,0.311029,0.0,29,0.311029,2,0.333333
4,0,11,0.000000,40,0.000000,0.0,32,0.311029,2,0.250000


Creating item/shop pair lags lag-based features

In [44]:
index_cols = ['shop_id', 'item_id', 'item_category_id', 'item_cat_id_fix', 'date_block_num']

cols_to_rename = list(all_data.columns.difference(index_cols))

print(cols_to_rename)

shift_range = [1, 2, 3, 4, 12]


for month_shift in tqdm(shift_range):

    train_shift = all_data[index_cols + cols_to_rename].copy()


    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift


    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x

    train_shift = train_shift.rename(columns=foo)


    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)



del train_shift

gc.collect()



all_data = all_data[all_data['date_block_num'] >= 12] # Don't use old data from year 2013

lag_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]]

all_data = downcast_dtypes(all_data)



['item_cat_id_fix_cnt_month_mean_Expanding', 'item_category_id_cnt_month_mean_Expanding', 'item_cnt_month', 'item_id_cnt_month_mean_Expanding', 'shop_id_cnt_month_mean_Expanding']


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:52<00:00, 10.46s/it]


In [45]:
all_data.head(10)

,date_block_num,item_cat_id_fix,item_cat_id_fix_cnt_month_mean_Expanding,item_category_id,item_category_id_cnt_month_mean_Expanding,item_cnt_month,item_id,item_id_cnt_month_mean_Expanding,shop_id,shop_id_cnt_month_mean_Expanding,...,item_cat_id_fix_cnt_month_mean_Expanding_lag_4,item_category_id_cnt_month_mean_Expanding_lag_4,item_cnt_month_lag_4,item_id_cnt_month_mean_Expanding_lag_4,shop_id_cnt_month_mean_Expanding_lag_4,item_cat_id_fix_cnt_month_mean_Expanding_lag_12,item_category_id_cnt_month_mean_Expanding_lag_12,item_cnt_month_lag_12,item_id_cnt_month_mean_Expanding_lag_12,shop_id_cnt_month_mean_Expanding_lag_12
3395293,12,11,0.242138,40,0.259059,0.0,30,2.634568,2,0.104104,...,0.242356,0.256372,0.0,3.750000,0.096310,0.000000,0.000000,0.0,0.000000,0.000000
3395294,12,11,0.242138,37,0.207829,0.0,31,1.849383,2,0.104103,...,0.242356,0.209389,0.0,2.628906,0.096309,0.000000,0.000000,0.0,0.000000,0.000000
3395295,12,11,0.242138,40,0.259058,1.0,32,2.450000,2,0.104102,...,0.242356,0.256372,0.0,2.890034,0.096307,0.000000,0.000000,0.0,0.311029,0.250000
3395296,12,11,0.242138,37,0.207828,1.0,33,0.802273,2,0.104112,...,0.242355,0.209388,0.0,0.810997,0.096306,0.000000,0.311029,1.0,0.311029,0.200000
3395297,12,11,0.242139,40,0.259059,0.0,34,0.209091,2,0.104121,...,0.242355,0.256371,0.0,0.268041,0.096304,0.333333,0.000000,0.0,0.311029,0.333333
3395298,12,11,0.242139,37,0.207831,0.0,36,0.123457,2,0.104120,...,0.242355,0.209387,0.0,0.132812,0.096303,0.000000,0.000000,0.0,0.000000,0.000000
3395299,12,11,0.242139,40,0.259059,0.0,37,0.257453,2,0.104119,...,0.242355,0.256371,0.0,0.268182,0.096301,0.000000,0.000000,0.0,0.000000,0.000000
3395300,12,11,0.242138,41,0.197394,0.0,39,0.052632,2,0.104118,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3395301,12,12,0.180442,57,0.120071,0.0,40,0.118182,2,0.104117,...,0.175414,0.120527,0.0,0.134021,0.096300,0.311029,0.311029,0.0,0.311029,0.250000
3395302,12,12,0.180442,57,0.120070,0.0,41,0.082192,2,0.104116,...,0.175414,0.120525,0.0,0.092784,0.096298,0.000000,0.000000,0.0,0.311029,0.222222


In [46]:
dates_train = train[['date', 'date_block_num']].drop_duplicates()

Seperate test data from dates_train

In [47]:
dates_test = dates_train[dates_train['date_block_num'] == 34-12]

In [48]:
dates_test['date_block_num'] = 34

C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
dates_test['date'] = dates_test['date'] + pd.DateOffset(years=1)

C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


concatenate train and test to convert date format.

In [50]:
dates_all = pd.concat([dates_train, dates_test])

In [51]:
dates_all['dow'] = dates_all['date'].dt.dayofweek

In [52]:
dates_all['year'] = dates_all['date'].dt.year

In [53]:
dates_all['month'] = dates_all['date'].dt.month

In [54]:
dates_all.head()

,date,date_block_num,dow,year,month
0,2013-01-02,0,2,2013,1
1,2013-01-10,0,3,2013,1
2,2013-01-04,0,4,2013,1
3,2013-01-19,0,5,2013,1
4,2013-01-31,0,3,2013,1


Create new feature 'days_of_month'

In [55]:
dates_all =pd.get_dummies(dates_all, columns=['dow'])

dow_col = ['dow_' + str(x) for x in range(7)]

date_features = dates_all.groupby(['year', 'month', 'date_block_num'])[dow_col].agg('sum').reset_index()

date_features['days_of_month'] = date_features[dow_col].sum(axis=1)

In [56]:
date_features['year'] = date_features['year'] - 2013

In [57]:
date_features = date_features[['month', 'year', 'days_of_month', 'date_block_num']]

Merge date features into all_data

In [58]:
all_data = all_data.merge(date_features, on = 'date_block_num', how = 'left')

In [59]:
date_columns = date_features.columns.difference(set(index_cols))

Seperate test and train

In [60]:
train = all_data[all_data['date_block_num']!= all_data['date_block_num'].max()]

test = all_data[all_data['date_block_num']== all_data['date_block_num'].max()]

Scale feature columns

In [61]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()



to_drop_cols = ['date_block_num']

feature_columns = list(set(lag_cols + index_cols + list(date_columns)).difference(to_drop_cols))



train[feature_columns] = sc.fit_transform(train[feature_columns])

test[feature_columns] = sc.transform(test[feature_columns])


C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\xiaom\Downloads\anaconda\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\xiaom\Downloads\anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [62]:
train.shape

(5251165, 38)

In [63]:
test.shape

(214200, 38)

### Modeling

In [64]:
test =test.drop(['date_block_num', 'item_cnt_month'], 1)

In [65]:
test.shape

(214200, 36)

In [66]:
y_train = train['item_cnt_month']

In [67]:
train2 =train.drop(['date_block_num', 'item_cnt_month'], 1)

In [68]:
train2.shape

(5251165, 36)

In [69]:
    preds = []
    from sklearn.linear_model import (LinearRegression, SGDRegressor)

    import lightgbm as lgb



    sgdr= SGDRegressor(

        penalty = 'l2' ,

        random_state = SEED )

    lgb_params = {

                 'feature_fraction': 0.75,

                  'metric': 'rmse',

                  'nthread':1,

                  'min_data_in_leaf': 2**7,

                  'bagging_fraction': 0.75,

                  'learning_rate': 0.03,

                  'objective': 'mse',

                  'bagging_seed': 2**7,

                  'num_leaves': 2**7,

                  'bagging_freq':1,

                  'verbose':0

                  }



    estimators = [sgdr]



    for estimator in estimators:

        estimator.fit(train2, y_train)

        pred_test = estimator.predict(test)

        preds.append(pred_test)


        
        print()
        
    

    estimator = lgb.train(lgb_params, lgb.Dataset(train2, label=y_train), 300)

    pred_test = estimator.predict(test)

    preds.append(pred_test)


    print()

C:\Users\xiaom\Downloads\anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [70]:
predict = pd.DataFrame(preds[1])

In [71]:
Y_test = predict[0].clip(0, 20)

In [73]:
test2 = get_csv('test.csv.gz')

In [74]:
sub = pd.DataFrame({'ID':test2.ID, 'item_cnt_month':Y_test}) 

In [75]:
sub.head(10)

,ID,item_cnt_month
0,0,0.922084
1,1,0.007407
2,2,1.232342
3,3,0.141312
4,4,0.027360
5,5,0.870002
6,6,1.014033
7,7,0.011854
8,8,1.062951
9,9,0.011942


In [76]:
sub.to_csv('submission7.csv',index=False)